In [1]:
import face_recognition
import cv2
import numpy as np
import os

from skimage.io import imread

In [2]:
BASE_DIR = "C:\\Users\\vansh\\Desktop\\i"
image_dir = os.path.join(BASE_DIR,"new")

In [3]:
video_capture = cv2.VideoCapture(0)

In [10]:
no_encode=[]
def get_encoded_faces():
    encoded = {}

    for dirpath, dnames, fnames in os.walk(image_dir):
        for f in fnames:
            if f.endswith(".jpg") or f.endswith(".png") or f.endswith(".jpeg") or f.endswith(".PNG") or f.endswith(".JPG") or f.endswith(".JPEG") :
                face = face_recognition.load_image_file("new/" + f)
                face = imread("new/" + f)
                if face_recognition.face_encodings(face):
                    encoding = face_recognition.face_encodings(face)[0]
                    encoded[f.split(".")[0]] = encoding
                else:
                    print("Unable to Encode: ", f)
                    no_encode.append(f)
            
    return encoded

In [13]:
import numpy as np

faces = get_encoded_faces()
known_face_encodings = list(faces.values())
known_face_names = list(faces.keys())

# Save
np.save('encoding.npy', faces) 

# Load
faces = np.load('encoding.npy',allow_pickle='TRUE').item()

print (faces)
known_face_encodings = list(faces.values())
known_face_names = list(faces.keys())

Unable to Encode:  Ranveer_Singh.jpg
Unable to Encode:  Sunny_Deol.jpg
{'Aamir': array([-2.25454774e-02,  1.19265914e-01,  1.08164120e-02, -7.67354816e-02,
       -7.67511800e-02,  1.89698078e-02,  2.43227463e-03, -1.13947935e-01,
        2.07481176e-01, -8.61592740e-02,  1.36723310e-01, -3.23817991e-02,
       -1.98021159e-01, -1.28277037e-02, -3.59277613e-02,  4.62603495e-02,
       -2.09451199e-01, -8.29093084e-02, -6.76345304e-02, -8.97437483e-02,
        7.24766776e-02,  4.98094968e-02,  2.35838145e-02,  1.15452111e-02,
       -1.43300876e-01, -2.76954412e-01, -1.21938728e-01, -5.06100524e-03,
        1.26740858e-01, -8.33378807e-02,  5.69904409e-03,  4.31997888e-02,
       -2.15364888e-01, -1.10096782e-01,  4.25222144e-03,  6.07513264e-02,
       -2.59345863e-02, -3.86997648e-02,  1.24332920e-01,  4.37851362e-02,
       -1.44704878e-01,  2.22684499e-02,  4.17518541e-02,  3.39684606e-01,
        2.29445070e-01,  1.24828979e-01, -6.23667985e-02, -4.81550321e-02,
        8.81863534e

In [16]:
def classify_face():
    
#     faces = get_encoded_faces()
#     known_face_encodings = list(faces.values())
#     known_face_names = list(faces.keys())

#     # Load
#     faces = np.load('encoding.npy',allow_pickle='TRUE').item()
    
#     known_face_encodings = list(faces.values())
#     known_face_names = list(faces.keys())
    
    # Initialize some variable
    face_locations = []
    face_encodings = []
    face_names = []
    process_this_frame = True

    while True:
        ret, frame = video_capture.read()
        
        small_frame = cv2.resize(frame, (0,0), None, fx=0.25, fy=0.25)

        rgb_small_frame = small_frame#[:,:,::-1]
        
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        
        name ="Not Detected"
        
        for encodeFace,faceLoc in zip(face_encodings, face_locations):
            matches = face_recognition.compare_faces(known_face_encodings,encodeFace,0.5)
            faceDis = face_recognition.face_distance(known_face_encodings, encodeFace)
            matchIndex = np.argmin(faceDis)
            
            
            if matches[matchIndex]:
                name = known_face_names[matchIndex]
                
            if not matches[matchIndex]:
                name ="Unknown"
                
            y1,x2,y2,x1 = faceLoc
            y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0),2)
            cv2.rectangle(frame,(x1,y2-35), (x2,y2), (0,255,0), cv2.FILLED)
            cv2.putText(frame, name, (x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),2)
            #for i, faceDis in enumerate(faceDis):
                #cv2.putText(frame, str(round(faceDis,2)), (x2-55,y2-6),cv2.FONT_HERSHEY_COMPLEX,0.8,(255,255,255),2 )
            #cv2.putText(frame, str(round(faceDis[1],2)), (x2-40,y2-6),cv2.FONT_HERSHEY_COMPLEX,0.8,(255,255,255),1)
                

        # Display Result
        cv2.imshow("Video", frame)
        cv2.imwrite('DetectedFaces/faces_detected.jpg', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print(str(round(faceDis,2)))
            print(name)
            break
        
        if name != "Not Detected":
            
            if name != 'Unknown':
                #print(str(round(faceDis,2)))
                print(name)
                video_capture.release()
                cv2.destroyAllWindows()
                return name
            else:
                print(name)
            break
            
            
    video_capture.release()
    cv2.destroyAllWindows()

In [17]:
Detected_face = classify_face()

Vanshit Kandoi


In [18]:
import csv

with open('DATA.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        #print(row[0])
        if row[0] == Detected_face:
            print(row)

['Vanshit Kandoi', '20BCE2667', '9310104495', 'O+']


In [19]:
for dirpath, dnames, fnames in os.walk(image_dir):
        print(fnames)
        break

['Abhinav Jaiswal.jpeg', 'Ben Afflek.jpg', 'Elton John.jpg', 'Jerry Seinfeld.jpg', 'Madonna.jpg', 'Mindy Kaling.jpg', 'Saswata Basu.jpg', 'Sneha Jayshri.png', 'Vanshit Kandoi.jpg']
